In [1]:
## Libraries
import json, os, re, datetime, csv
from pathlib import Path
import pandas as pd
import nltk
from nltk.corpus import stopwords

### We have two datasets:
#### 1) Linkedin jobs ads (dynamic)
#### 2) ECSF (static)

The first dataset, has to be dynamic in other words, we need to put it in a producer-topic-consumer-casssandra

The second dataset, has to be in cassandra

### To clean the linkedin job ads dataset:

#### 1) delete irrelevent columns
#### 2) remove duplicates
#### 3) remove null values
#### 4) preprocess Skills column
#### 5) preprocess Description column
#### 6) preprocess Title column

In [ ]:
df = pd.read_json('jobs.json')
# number of rows / samples
row_count = df.shape[0]
print(f"Number of rows in the dataset: {row_count}")
df.head()

In [6]:
df.columns

Index(['Title', 'Description', 'Primary Description', 'Detail URL', 'Location',
       'Skill', 'Insight', 'Job State', 'Poster Id', 'Company Name',
       'Company Logo', 'Created At', 'Scraped At'],
      dtype='object')

#### Check for duplicates

In [50]:
# Duplicate check (first 5 columns)
subset_cols = df.columns[:5].tolist()
initial_rows = df.shape[0]

duplicate_count = df.duplicated(subset=subset_cols).sum()
print(f"number of detected duplicates: {duplicate_count}")


number of detected duplicates: 0


#### Drop unrelated columns (meaningless columns)

In [51]:
# Drop hiring manager-related columns, scrape info, non needed columns
columns_to_drop = [
    'Insight', 'Job State', 'Poster Id','Company Logo', 'Created At', 'Scraped At',
]
df.drop(columns=columns_to_drop, errors="ignore", inplace=True)

df.columns

Index(['Title', 'Description', 'Primary Description', 'Detail URL', 'Location',
       'Skill', 'Company Name'],
      dtype='object')

#### Cleaning Description column

In [52]:
print('-> The first row of the Description column before cleaning: \n',df["Description"][1][:500])
# 1) remove \n
df["Description"] = df["Description"].str.replace("\n", ". ", regex=True)


# 2) remove unrelated sentences
# List of unwanted phrases to remove
unwanted_phrases = [
    "job description", "job title", "role description", "about the job",
    "about the role", "about us", "about the opportunity", "requirements",
    "job requirements", "role requirements", "your role", "your job",
    "offer", "employment offer", "your profile", "responsibilities",
    "job responsibilities", "role responsibilities", "overview", "position overview", 
    "who are we?", "who we are", "who are we ?"
]
# Precompile regex patterns for efficiency
patterns = [re.compile(rf"^{phrase}[:\s]*", re.IGNORECASE) for phrase in unwanted_phrases]

# Cleaning function
def clean_description(text):
    if isinstance(text, str):
        for pattern in patterns:
            text = pattern.sub("", text).strip()  # Remove matched phrase and trim spaces
        return text
    return ""

# Apply function to the description column

df["Description"] = df["Description"].apply(clean_description)
print('---------------------------------------------------------')
print('-> The first row of the Description column after cleaning: \n',df["Description"][1][:500])

-> The first row of the Description column before cleaning: 
 Vattenfall IT’s Data Centre department manages on-premises workloads and Public Cloud services under a hybrid strategy. While the Public Cloud Platform Services team handles cloud-based workloads, the Data Centre team oversees country-based Data Centers and is transitioning from traditional hosting to Virtual Private Cloud (VPC) platforms to meet regulatory and business-specific needs.

Private Cloud Platform Services

A new department within the Data Centre organization manages Private Cloud Pl
---------------------------------------------------------
-> The first row of the Description column after cleaning: 
 Vattenfall IT’s Data Centre department manages on-premises workloads and Public Cloud services under a hybrid strategy. While the Public Cloud Platform Services team handles cloud-based workloads, the Data Centre team oversees country-based Data Centers and is transitioning from traditional hosting to Virtual Private

#### Clean Skills column

In [53]:
print('-> The first row of the Description column before cleaning: \n',df["Skill"][1][:400])

# Preprocess the Skill column
def clean_skills(skill_str):
    if isinstance(skill_str, str):
        # Remove "Skills: " if it appears at the beginning
        skill_str = re.sub(r"^Skills:\s*", "", skill_str).strip()
        # Remove "X of Y skills match your profile - you may be ..." pattern
        skill_str = re.sub(r"\d+\s+of\s+\d+\s+skills match your profile - you may be.*", "", skill_str, flags=re.IGNORECASE).strip()
        # Remove ", +X more" where X is any number
        skill_str = re.sub(r",\s\+\d+\s+more", "", skill_str).strip()
        return skill_str
    return ""

df["Skill"] = df["Skill"].apply(clean_skills)
df.head(5)

print('---------------------------------------------------------')
print('-> The first row of the Description column after cleaning: \n',df["Skill"][1][:400])


-> The first row of the Description column before cleaning: 
 Skills: English, Data Centers, +8 more
---------------------------------------------------------
-> The first row of the Description column after cleaning: 
 English, Data Centers


#### Clean Title

In [54]:
#1) download the stopwords
nltk.download("stopwords")

# 2) Load stopwords for each detected language
stop_words = set()
languages = ["english", "german", "dutch", "italian", "spanish", "french", "portuguese"]
# Add stopwords for each language to the set
for lang in languages:
    stop_words.update(set(stopwords.words(lang)))
# Add custom stopwords to the set
custom_stopwords = {"mwd", "mfd", "gender", "fh", "mf", "hf", "fmd", "wmd", "etc", "M/F"}
stop_words.update(custom_stopwords)

# 3) Function to clean stop words from text 
def clean_text(text):
    if isinstance(text, str):
        text = text.lower()  # Convert to lowercase
        text = re.sub(r"\d+", "", text)  # Remove numbers
        text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
        text = " ".join(word for word in text.split() if word not in stop_words)  # Remove stopwords
        return text
    return ""

# Clean "Title" and "Description" columns
df["Title"] = df["Title"].apply(clean_text)
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samiha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Title,Description,Primary Description,Detail URL,Location,Skill,Company Name
0,inhouse consultant information security awareness,". As a self-financed family company, we are mo...","Deichmann · Essen, North Rhine-Westphalia, Ger...",https://www.linkedin.com/jobs/view/4300643770,"Essen, North Rhine-Westphalia, Germany","Information Security, Phishing",Deichmann
1,senior storage engineer,Vattenfall IT’s Data Centre department manages...,"Vattenfall · Stockholm, Stockholm County, Swed...",https://www.linkedin.com/jobs/view/4279274865,"Stockholm, Stockholm County, Sweden","English, Data Centers",Vattenfall
2,dba,SoSafe has the ambition to become the leading ...,"SoSafe · Madrid, Community of Madrid, Spain (O...",https://www.linkedin.com/jobs/view/4292576067,"Madrid, Community of Madrid, Spain","Amazon Web Services (AWS), Amazon Relational D...",SoSafe
3,business system analyst,"At Exness, we are not just a leading trading b...","Exness · Limassol, Cyprus (Hybrid)",https://www.linkedin.com/jobs/view/4272079301,"Limassol, Cyprus","English, Programming Languages",Exness
4,data privacy protection consultant hmx,"In Experites, we seek privacy and data protect...","EXPERTIS SPAIN · Three songs, Community of Mad...",https://www.linkedin.com/jobs/view/4300652034,"Tres Cantos, Community of Madrid, Spain","Data Privacy, Data Protection Act",Experis España


#### Clean Location column and add Country column

In [ ]:
# ------------------------------------------------------------------
# 1. mapping dictionary
# ------------------------------------------------------------------
df["Location"] = df["Location"].replace({

    "England Metropolitan Area": "United Kingdom",
    "Brno Metropolitan Area": "Czechia", "Plzen Metropolitan Area": "Czechia", "Prague Metropolitan Area": "Czechia",
    "Sofia Metropolitan Area": "Bulgaria", "Plovdiv Metropolitan Area": "Bulgaria",
    "Athens Metropolitan Area": "Greece", "Thessaloniki Metropolitan Area": "Greece",
    "Geneva Metropolitan Area": "Switzerland", "Basel Metropolitan Area": "Switzerland",
    "Zürich Metropolitan Area": "Switzerland", "Lugano Metropolitan Area": "Switzerland",
    "Lausanne Metropolitan Area": "Switzerland", "Lucerne Metropolitan Area": "Switzerland",
    "Sankt Gallen Metropolitan Area": "Switzerland",
    "Greater Turin Metropolitan Area": "Italy", "Italy Metropolitan Area": "Italy",
    "Greater Pavia Metropolitan Area": "Italy", "Greater Parma Metropolitan Area": "Italy",
    "Greater Naples Metropolitan Area": "Italy", "Greater Bergamo Metropolitan Area": "Italy",
    "Greater Brescia Metropolitan Area": "Italy", "Greater Verona Metropolitan Area": "Italy",
    "Greater Rennes Metropolitan Area": "France", "Greater Grenoble Metropolitan Area": "France",
    "Greater Bordeaux Metropolitan Area": "France", "Greater Montpellier Metropolitan Area": "France",
    "Greater Strasbourg Metropolitan Area": "France", "Greater Toulouse Metropolitan Area": "France",
    "Greater Nantes Metropolitan Area": "France",
    "Greater Malmö Metropolitan Area": "Sweden", "Greater Gothenburg Metropolitan Area": "Sweden",
    "Greater Helsingborg Metropolitan Area": "Sweden", "Greater Västerås Metropolitan Area": "Sweden",
    "Tarnow Metropolitan Area": "Poland", "Cracow Metropolitan Area": "Poland",
    "Kielce Metropolitan Area": "Poland", "Katowice Metropolitan Area": "Poland",
    "Lodz Metropolitan Area": "Poland", "Poznan Metropolitan Area": "Poland",
    "Zamosc Metropolitan Area": "Poland",
    "Greater Palma de Mallorca Metropolitan Area": "Spain", "Greater Bilbao Metropolitan Area": "Spain",
    "Greater Vigo Metropolitan Area": "Spain", "Greater Granada Metropolitan Area": "Spain",
    "Greater Valencia Metropolitan Area": "Spain",
    "Galway Metropolitan Area": "Ireland",
    "Kortrijk Metropolitan Area": "Belgium", "Antwerp Metropolitan Area": "Belgium",
    "Ghent Metropolitan Area": "Belgium", "Charleroi Metropolitan Area": "Belgium",
    "Greater Nuremberg Metropolitan Area": "Germany",
    "Pecs Metropolitan Area": "Hungary",
    "Ljubljana Metropolitan Area": "Slovenia",

    # === NEW ONES YOU JUST POSTED ===
    "Hannover-Braunschweig-Göttingen-Wolfsburg Region": "Germany",
    "Brabantine City Row": "Netherlands",
    "Greater Dusseldorf Area": "Germany",
    "Greater Mulhouse Area": "France",
    "Linz-Wels-Steyr Area": "Austria",
    "Greater Alicante Area": "Spain",
    "Greater Funchal Area": "Portugal",
    "Greater Ipswich Area": "United Kingdom",
    "Greater La Coruña Area": "Spain",
    "Eindhoven Area": "Netherlands",
    "Greater Liverpool Area": "United Kingdom",
    "Greater Kecskemet Area": "Hungary",
    "Greater Leipzig Area": "Germany",
    "Utrecht Area": "Netherlands",
    "Greater Kiel Area": "Germany",
    "Greater Lyon Area": "France",
    "Greater Bielefeld Area": "Germany",
    "Greater Avignon Area": "France",
    "Greater Bern Area": "Switzerland",
    "Greater Hamburg Area": "Germany",
    "Greater Leeds Area": "United Kingdom",
    "Greater Metz Area": "France",
    "Greater Trencin Area": "Slovakia",
    "European Economic Area": "European Economic Area",
    'Osnabrück Land': 'Germany',
    "Greater Salzburg": "Austria",
    "Greater Sankt Polten": "Austria",      # also works if written without ö
    "Greater Graz": "Austria",
    "Greater Dublin": "Ireland",
    "Greater Nottingham": "United Kingdom",
})

# ------------------------------------------------------------------
# Create a new column 'Country' by extracting the text after the last comma
df["Country"] = df["Location"].apply(lambda x: x.split(",")[-1].strip() if "," in x else x.strip())

# Exclude rows where Location contains "EMEA" or "European Union" (as these are not countries)
df = df[~df["Location"].str.contains("EMEA|European Union", na=False)]

In [ ]:
# Check remaining "area"
country_df = df[df['Country'].str.contains('great', case=False, na=False)]
country_df['Country']

#### Save the df

In [ ]:
df.to_json('cleaned_linkedin_jobs.json', orient='records', lines=False, indent=4)

In [ ]:
df.to_csv('cleaned_linkedin_jobs.csv', index=False)